In [72]:
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras import layers
## Plotly

import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
# Others
import nltk
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords

from sklearn.manifold import TSNE
import re
#nltk
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.tokenize import WordPunctTokenizer

#genism
import gensim
from gensim.models import Word2Vec, KeyedVectors

In [57]:
cols=['tweet','tag','n']
df=pd.read_csv("./FinalTweetList.csv",names=cols)
df=df.drop(df.columns[2],axis=1)
df

,tweet,tag
0,@ICICIBank_Care Bank employees of 21 c Faridab...,displeasure
1,@AxisBankSupport 18..233.... is permanently bu...,displeasure
2,"@TheOfficialSBI 3 hours in line,still waiting....",displeasure
3,@HDFCBank_Cares Both the ATM machines here in ...,displeasure
4,@ICICIBank Thanks icici bank I got case in you...,miscellaneous
...,...,...
5689,@HDFCBank_Cares over 25 calls this week.I gues...,displeasure
5690,@ICICIBank_Care e-Pay slip as system not worki...,displeasure
5691,@TheOfficialSBI staff at Sbi Akaltara branch C...,miscellaneous
5692,"@singhraviranja4 @AxisBankSupport Hi, sorry fo...",miscellaneous


In [58]:
def remove_stopwords(tweet):
    stop_words = set(stopwords.words('english')) 
    word_tokens = word_tokenize(tweet) 
    filtered_sentence = [w for w in word_tokens if not w in stop_words] 
    #print(" ".join(filtered_sentence))
    return " ".join(filtered_sentence)
#function for cleaning the tweets
def clean(tweet):
    tweet=tweet.lower()
    tweet=tweet.lstrip()
    tweet=tweet.rstrip()
    tweet=remove_stopwords(tweet)
    negations_dic = {"isn't":"is not", "aren't":"are not", "wasn't":"was not", "weren't":"were not",
                "haven't":"have not","hasn't":"has not","hadn't":"had not","won't":"will not",
                "wouldn't":"would not", "don't":"do not", "doesn't":"does not","didn't":"did not",
                "can't":"can not","couldn't":"could not","shouldn't":"should not","mightn't":"might not",
                "mustn't":"must not"}
    neg_pattern = re.compile(r'\b(' + '|'.join(negations_dic.keys()) + r')\b')
    neg_handled = neg_pattern.sub(lambda x: negations_dic[x.group()], tweet)
    letters_only = re.sub("[^a-zA-Z]", " ", neg_handled)
    # During the letters_only process two lines above, it has created unnecessay white spaces,
    # I will tokenize and join together to remove unneccessary white spaces
    words = [x for x  in WordPunctTokenizer().tokenize(letters_only) if len(x) > 1]
    return " ".join(words)

In [59]:
df['tweet'] = df['tweet'].map(lambda x: clean(x))
df["tag"]

0         displeasure
1         displeasure
2         displeasure
3         displeasure
4       miscellaneous
            ...      
5689      displeasure
5690      displeasure
5691    miscellaneous
5692    miscellaneous
5693      displeasure
Name: tag, Length: 5694, dtype: object

In [60]:
labels=[]
for label in df["tag"]:
    if(label)=="displeasure":
        labels.append(0)
    elif(label=="miscellaneous"):
        labels.append(1)
    elif(label=="compliment"):
        labels.append(2)
    else:
        print("wrong")


In [61]:
vocabulary_size=0
for i in df["tweet"]:
    for word in i.split(" "):
        if word not in l:
            vocabulary_size=vocabulary_size+1


In [62]:
### Create sequence
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(df['tweet'])
sequences = tokenizer.texts_to_sequences(df['tweet'])
data = pad_sequences(sequences, maxlen=50)

In [64]:
## Network architecture
model = Sequential()
model.add(Embedding(len(l), 100, input_length=50))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
## Fit the model
model.fit(data, np.array(labels), validation_split=0.086, epochs=3)

/anaconda3/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning:

Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.



Train on 5204 samples, validate on 490 samples
Epoch 1/3
5204/5204 [==============================] - 14s 3ms/step - loss: 0.4713 - accuracy: 0.4800 - val_loss: -0.2663 - val_accuracy: 0.5918
Epoch 2/3
5204/5204 [==============================] - 13s 2ms/step - loss: -0.4728 - accuracy: 0.5790 - val_loss: -1.2853 - val_accuracy: 0.5286
Epoch 3/3
5204/5204 [==============================] - 12s 2ms/step - loss: -1.4689 - accuracy: 0.5742 - val_loss: -2.1866 - val_accuracy: 0.5265


In [77]:
X = df.tweet
y = df.tag
train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.0878, random_state = 42)

NameError: name 'train_test_split' is not defined

In [66]:
embeddings_index = dict()
f = open('embedded.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [67]:
embedding_matrix = np.zeros((vocabulary_size, 100))
for word, index in tokenizer.word_index.items():
    if index > vocabulary_size - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

In [70]:
## create model
model_glove = Sequential()
model_glove.add(Embedding(vocabulary_size, 100, input_length=50, weights=[embedding_matrix], trainable=False))
model_glove.add(Dropout(0.2))
model_glove.add(Conv1D(64, 5, activation='relu'))
model_glove.add(MaxPooling1D(pool_size=4))
model_glove.add(LSTM(100))
model_glove.add(Dense(1, activation='sigmoid'))
model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
## Fit train data
model_glove.fit(data, np.array(labels), validation_split=0.86, epochs = 3)


Train on 797 samples, validate on 4897 samples
Epoch 1/3
797/797 [==============================] - 2s 3ms/step - loss: 0.6893 - accuracy: 0.4705 - val_loss: 0.6552 - val_accuracy: 0.3923
Epoch 2/3
797/797 [==============================] - 1s 2ms/step - loss: 0.6496 - accuracy: 0.4843 - val_loss: 0.6173 - val_accuracy: 0.4789
Epoch 3/3
797/797 [==============================] - 1s 2ms/step - loss: 0.5706 - accuracy: 0.5721 - val_loss: 0.5454 - val_accuracy: 0.4919


In [74]:
embedding_dim = 100

model = Sequential()
model.add(layers.Embedding(vocabulary_size, embedding_dim, weights=[embedding_matrix], trainable=False))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, None, 100)         51300     
_________________________________________________________________
conv1d_3 (Conv1D)            (None, None, 128)         64128     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 10)                1290      
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 11        
Total params: 116,729
Trainable params: 65,429
Non-trainable params: 51,300
_________________________________________________________________


In [75]:
model_glove.fit(data, np.array(labels), validation_split=0.86, epochs = 3)

Train on 797 samples, validate on 4897 samples
Epoch 1/3
797/797 [==============================] - 1s 2ms/step - loss: 0.5410 - accuracy: 0.5659 - val_loss: 0.5126 - val_accuracy: 0.5289
Epoch 2/3
797/797 [==============================] - 1s 2ms/step - loss: 0.4319 - accuracy: 0.6198 - val_loss: 0.5064 - val_accuracy: 0.4864
Epoch 3/3
797/797 [==============================] - 1s 2ms/step - loss: 0.3566 - accuracy: 0.6136 - val_loss: 0.4853 - val_accuracy: 0.5542


In [76]:
model.predict(test_x)

NameError: name 'test_x' is not defined